In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,median_absolute_error, mean_squared_log_error
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense ,Dropout, Embedding, LSTM, Bidirectional,GRU
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [3]:
def Scale (y_train,y_test):  
    train=y_train.to_frame()
    test= y_test.to_frame()
    scalerr = MinMaxScaler(feature_range=(0, 1))
    scaler = scalerr.fit(train)
    y_trainS =scaler.transform(train)
    y_testS = scaler.transform(test)
    return(y_trainS,y_testS,scaler)

def Create_Dataset (X, look_back):
    #create windows to observe the time series data
    Xs, ys = [], []
    
    for i in range(len(X)-look_back):
        v = X[i:i+look_back]
        Xs.append(v)
        ys.append(X[i+look_back])
        
    return np.array(Xs), np.array(ys)

In [7]:
data = pd.read_csv('dataset\AirPassengers.csv', usecols=[1], engine='python')
data = data.astype('float32')

In [29]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaler_fit = scaler.fit(data)
df_norm = scaler.fit_transform(data)
# split into train and test sets
train_size = int(len(df_norm) * 0.8)
test_size = len(df_norm) - train_size
train, test = df_norm[0:train_size,:], df_norm[train_size:len(df_norm),:]

Since LSTM, GRU, and BiLSTM algorithms require a 3D input shape (Batch-size, Time_steps (look back), input_dim (n_features)), we need a helper function, create_dataset, to reshape the input.

n_features is **1** in case of univariant time series data

“In this project, we define look_back = 12. This indicates that the model makes predictions based on data from the last 12 months. During the creation of training examples in the create_dataset function, the input for each iteration includes data from the first 12 months, and the corresponding output is the value for the 12th month.

In [24]:
look_back =12
x_train, y_train =Create_Dataset (train, look_back)
x_test, y_test = Create_Dataset (test, look_back)

In [25]:
print(train.shape)
print(x_train.shape)
print(y_train.shape)
print(test.shape)
print(x_test.shape)
print(y_test.shape)

(115, 1)
(103, 12, 1)
(103, 1)
(29, 1)
(17, 12, 1)
(17, 1)


choosing the number of hidden layers:

- Well if the data is linearly separable then you don’t need any hidden layers at all.
- If data is less complex and is having fewer dimensions or features then neural networks with 1 to 2 hidden layers would work.
- If data is having large dimensions or features then to get an optimum solution, 3 to 5 hidden layers can be used.

The final layer distribution (LSTM/GRU/BILSTM) consists of: two LSTM layers and one output layer with a single unit (only one characteristic is expected, i.e. the amount of revenue expected, so the output layer will have only one unit)(Dense(1)).

In [59]:
def Train_LSTM(X_trainn,y_trainn,units,batch_size,epochs):
    #==Define model architecture
    model = Sequential()
    #===== Add LSTM layers
    model.add(LSTM(units = units, return_sequences=True,activation='relu',
                   input_shape=(X_trainn.shape[1], X_trainn.shape[2])))
    #===== Hidden layer
    model.add(LSTM(units = units, return_sequences=True))
    model.add(LSTM(units = units, return_sequences=True))
    #=== output layer
    model.add(Dense(units = 1))
    #==== Compiling the model
    model.compile(optimizer='adam', loss='mape') 
    #====== Fit Model
    early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 10)
    history = model.fit(X_trainn, y_trainn, epochs = epochs, validation_split = 0.2,
                        batch_size = batch_size, shuffle = False, callbacks = [early_stop],verbose=0)
    
    modelN='LSTM'
    return(history,modelN,model)

def Train_BiLSTM(X_trainn,y_trainn,units,batch_size,epochs):
    #==Define model architecture    
    model = Sequential()
    #===== Add LSTM layers
    model.add(Bidirectional(LSTM(units = units, return_sequences=True,activation='relu',
                            input_shape=(X_trainn.shape[1], X_trainn.shape[2]))))
    #===== Hidden layer
    model.add(Bidirectional(LSTM(units = units,return_sequences=True)))
    model.add(Bidirectional(LSTM(units = units,return_sequences=True)))
    #=== output layer
    model.add(Dense(1))
    #==== Compiling the model
    model.compile(optimizer='adam', loss='mape') 
    #====== Fit Model
    early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 10)
    history = model.fit(X_trainn, y_trainn, epochs = epochs, validation_split = 0.2,
                        batch_size = batch_size, shuffle = False, callbacks = [early_stop],verbose=0)
    
    modelN='BiLSTM'
    return(history,modelN,model)

def Train_GRU(X_trainn,y_trainn,units,batch_size,epochs):
    #==Define model architecture 
    model = Sequential()
    #===== Add LSTM layers
    model.add(GRU (units = units, return_sequences = True,activation='relu',
                   input_shape = [X_trainn.shape[1], X_trainn.shape[2]]))
    #model.add(Dropout(0.2)) 
    #===== Hidden layer
    model.add(GRU(units = units,return_sequences = True))  
    model.add(GRU(units = units,return_sequences = True)) 
    model.add(Dropout(0.3))
    #=== output layer
    model.add(Dense(units = 1)) 
    #==== Compiling the model
    model.compile(optimizer='adam', loss='mape') 
    #====== Fit Model
    early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 10)
    history = model.fit(X_trainn, y_trainn, epochs = epochs, validation_split = 0.2,batch_size = batch_size,
                        shuffle = False, callbacks = [early_stop],verbose=0)
    
    modelN='GRU'
    return(history,modelN,model)

def make_pred_LSTM(model,scaled_train_data,scaled_test_data,n_input,n_features,scalerfit):
    
    #=========== Predict train =============#
    lstm_predictions_scaledt = list()
    batcht = scaled_train_data[:n_input]
    current_batcht = batcht.reshape((1, n_input, n_features))
    for i in range(len(scaled_train_data)):   
        lstm_predt = model.predict(current_batcht)[0]
        lstm_predictions_scaledt.append(lstm_predt) 
        current_batcht = np.append(current_batcht[:,1:,:],[lstm_predt],axis=1)
    lstm_predict_train = abs(scalerfit.inverse_transform(lstm_predictions_scaledt))
    
    #============ Predict test ==============#
    lstm_predictions_scaled = list()
    batch = scaled_train_data[-n_input:]
    #current_batch = batch.reshape((1, n_input, n_features))
    current_batch = batch
    for i in range(len(scaled_test_data)):   
        lstm_pred = model.predict(current_batch)[0]
        lstm_predictions_scaled.append(lstm_pred) 
        current_batch = np.append(current_batch[:,1:,:],[lstm_pred],axis=1)
    lstm_predict_test = abs(scalerfit.inverse_transform(lstm_predictions_scaled))
    
    return (abs(lstm_predict_train),abs(lstm_predict_test))

def make_Forecast_LSTM(model,scaled_test_data,n_input,n_features,scalerfit,nbr_month):
    lstm_predictions_scaled = list()
    batch = scaled_test_data[-n_input:]
    print(batch.shape)
    #current_batch = batch.reshape((1, n_input, n_features))
    current_batch = batch
    for i in range(nbr_month+1):   
        lstm_pred = model.predict(current_batch)[0]
        lstm_predictions_scaled.append(lstm_pred) 
        current_batch = np.append(current_batch[:,1:,:],[lstm_pred],axis=1)
    lstm_forcast = scalerfit.inverse_transform(lstm_predictions_scaled)
    return (abs(lstm_forcast))

In [34]:
history, name, model = Train_BiLSTM(x_train, y_train, 100, 16, 10)

In [41]:
predict_train = model.predict(x_train)
predict_test = model.predict(x_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 736ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


In [46]:
trainPredict = scaler.inverse_transform(predict_train.reshape(-1, 1))
trainY = scaler.inverse_transform(y_train.reshape(-1, 1))
testPredict = scaler.inverse_transform(predict_test.reshape(-1, 1))
testY = scaler.inverse_transform(y_test.reshape(-1, 1))

In [50]:
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 13.30 RMSE
Test Score: 202.91 RMSE


In [60]:
forecast =make_Forecast_LSTM(model, x_test, x_test.shape[1], x_test.shape[2], scaler_fit, 12)

(12, 12, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 12 and the array at index 1 has size 1

In [54]:
x_test.shape[1]

12